In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import os
import torch.nn.functional as F

In [ ]:

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"GPU: {torch.cuda.get_device_name()}")
else:
    device = 'cpu'

In [ ]:

x = torch.load('train/x_train.pt')
y = torch.load('train/y_train.pt')
vocab = torch.load('train/vocab.pt')

x.shape, y.shape, len(vocab)

In [ ]:
class Code2vec(nn.Module):
    def __init__(self, vocab_size, embedding_size):
        super(Code2vec, self).__init__()


        self.embed_layer = nn.Embedding(vocab_size, 512, padding_idx=2)

        self.final_layer = nn.Linear(2048, embedding_size)

        self.layers = nn.Sequential(
 
            nn.Conv1d(512, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(256, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Flatten(),
            nn.Linear(4096, 2048),
            nn.ReLU(),
        )

        self.residual_layer = nn.Sequential(
            nn.MaxPool1d(64),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32768, 2048),
            nn.ReLU()
        )

    def forward(self, x):

        x = self.embed_layer(x)
        x = x.permute(0, 2, 1) # 1, 256, 4096

        residual = x


        x = self.layers(x) + self.residual_layer(residual)

        x = self.final_layer(x)
        return x

In [ ]:

    
dataset = TensorDataset(x,y)
BATCH_SIZE = 256

train_dataset, test_dataset = random_split(dataset, [0.95, 0.05])

loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

model = Code2vec(len(vocab), 512)
model = model.to(device)


def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CosineEmbeddingLoss()

epochs = 10

In [ ]:
train_accuracies = []
train_losses = []
test_accuracies = []
current_epoch = 0

def calculate_accuracy(model, dataset, device, batch_size=BATCH_SIZE):

    model.eval()

    correct = 0
    total = 0
    
    test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            pair_code1 = batch_x[:, 0, :].to(device)
            pair_code2 = batch_x[:, 1, :].to(device)
            batch_y = batch_y.to(device).float()
            
            cosine_sim = F.cosine_similarity(
                model(pair_code1), 
                model(pair_code2), 
                dim=1
            )
            
            predictions = torch.where(cosine_sim >= 0.8, 1.0, -1.0)
            
            correct += (predictions == batch_y).sum().item()
            total += batch_y.size(0)
    
    model.train()
    return correct / total * 100


In [ ]:
from tqdm.auto import tqdm

epoch_bar = tqdm(range(epochs), desc='Training')

for epoch in epoch_bar:
    running_loss = 0.0
    epoch_loss = 0.0
    
    batch_bar = tqdm(enumerate(loader), 
                    total=len(loader), 
                    desc=f'Epoch {current_epoch + epoch + 1}',
                    leave=False)
    
    for i, batch in batch_bar:
        batch_x, batch_y = batch
        pair_code1 = batch_x[:, 0, :].to(device)
        pair_code2 = batch_x[:, 1, :].to(device)
        batch_y = batch_y.to(device).float()

        pair1_output = model(pair_code1)
        pair2_output = model(pair_code2)
        
        t = batch_y.clone()
        
        loss = criterion(pair1_output, pair2_output, t)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        epoch_loss += loss.item()
        
        batch_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Avg': f'{running_loss/(i+1):.4f}'
        })
    
    avg_epoch_loss = epoch_loss / len(loader)
    train_losses.append(avg_epoch_loss)  # 손실 기록
    
    train_acc = calculate_accuracy(model, train_dataset, device)
    test_acc = calculate_accuracy(model, test_dataset, device)
    train_accuracies.append(train_acc)
    test_accuracies.append(test_acc)
    print(f'Epoch {current_epoch + epoch + 1}: Train Accuracy: {train_acc:.2f}% | Test Accuracy: {test_acc:.2f}%')
    

    epoch_bar.set_postfix({
        'Epoch Loss': f'{avg_epoch_loss:.4f}',
        'Train Acc': f'{train_acc:.2f}',
        'Test Acc': f'{test_acc:.2f}'
    })

current_epoch += epochs


print("Training completed!")
print(f"Total epochs trained: {current_epoch}")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 3))

plt.subplot(1, 3, 1)
plt.plot(range(1, len(train_losses) + 1), train_losses, 'b-')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)

plt.subplot(1, 3, 2)
if train_accuracies and test_accuracies:
    epochs_range = range(1, len(train_accuracies) + 1)
    plt.plot(epochs_range, train_accuracies, 'g-', marker='o', label='Train Accuracy')
    plt.plot(epochs_range, test_accuracies, 'r-', marker='s', label='Test Accuracy')
    plt.title('Train vs Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True)

plt.subplot(1, 3, 3)
ax1 = plt.gca()
ax2 = ax1.twinx()

ax1.plot(range(1, len(train_losses) + 1), train_losses, 'b-', label='Train Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss', color='b')
ax1.tick_params(axis='y', labelcolor='b')

if test_accuracies:
    ax2.plot(range(1, len(test_accuracies) + 1), test_accuracies, 'r-', marker='o', label='Test Accuracy')
    ax2.set_ylabel('Test Accuracy (%)', color='r')
    ax2.tick_params(axis='y', labelcolor='r')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels() if test_accuracies else ([], [])
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.title('Loss and Accuracy Over Time')
plt.tight_layout()
plt.show()


if train_losses:
    print(f"final loss: {train_losses[-1]:.4f}")
if train_accuracies:
    print(f"train acc: {train_accuracies[-1]:.2f}%")
if test_accuracies:
    print(f"test acc: {test_accuracies[-1]:.2f}%")


In [ ]:
torch.save(model, 'model.pth')